In [ ]:
import gc
gc.collect()

778

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os
# from scipy.misc import imread, imresize
# this line was added since the above line was throwing an error
import cv2 as cv
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
!unzip /content/gdrive/MyDrive/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00029.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00031.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00033.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00035.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00037.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00039.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00041.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00043.png  
 extracting: Project_data/train/WIN_20180926_17_33_08

In [ ]:
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [1,8,9,10,11,13,14,15,16,19,20,21,24,26,28]#create a list of image numbers you want to use for a particular video
    
    while True:
        t = np.random.permutation(folder_list)
        # print("len(t) before for", len(t))
        num_batches = len(t)//batch_size# calculate the number of batches
        # print("num_batches before first for",num_batches)
        for batch in range(num_batches): # we iterate over the number of batches
            # print("batch inside for",batch+1)
            batch_data = np.zeros((batch_size,15,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                # print("fetching folder number", folder + (batch*batch_size))
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #the images were of 2 sizes a sample of each were viewed individually and resized
                    if image.shape[1] == 160:
                      # crop_img = image[y:y+h, x:x+w]
                      crop_img = image[40:,40:].astype(np.float32)
                    else:
                      crop_img = image[100:,100:].astype(np.float32)
                  
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = cv.resize(image,(84,84))
                    #Normalizing the image
                    image = image - np.percentile(image,5)/np.percentile(image,95) - np.percentile(image,5)
                    batch_data[folder,idx,:,:,0] = image[:,:,0]#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                # print(batch_labels)
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

       
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size) != 0 and batch == num_batches-1:
            batch=0
            remaining = len(t)%batch_size
            # print("remaining", remaining)
            batch_data = np.zeros((remaining,15,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((remaining,5)) # batch_labels is the one hot representation of the output
            for folder in range(remaining): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                # print("fetching folder number", folder + (num_batches*batch_size))
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    if image.shape[1] == 160:
                      # crop_img = image[y:y+h, x:x+w]
                      crop_img = image[40:,40:].astype(np.float32)
                    else:
                      crop_img = image[100:,100:].astype(np.float32)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = cv.resize(crop_img,(84,84))
                    image = image - np.percentile(image,5)/np.percentile(image,95) - np.percentile(image,5)
                    batch_data[folder,idx,:,:,0] = image[:,:,0]#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]#normalise and feed in the image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers,regularizers


# #write your model here

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(15,84,84,3),kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides= (2,2,1)))


model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same',kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides= (2,2,1)))

# Dropout
model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same',kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides= (2,2,1)))

model.add(Flatten())
# Dropout
model.add(Dropout(0.20))

model.add(Dense(512, activation='elu',kernel_regularizer=regularizers.l2(0.01)))
# Dropout
model.add(Dropout(0.25))
model.add(Dense(512, activation='elu',kernel_regularizer=regularizers.l2(0.01)))
# Dropout
model.add(Dropout(0.20))
model.add(Dense(5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
# optimiser = 'Adam'#write your optimizer
sgd = optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
rms = tf.keras.optimizers.RMSprop(
    learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop')
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 15, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 84, 84, 64)    256       
_________________________________________________________________
activation_12 (Activation)   (None, 15, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 7, 42, 84, 64)     0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 7, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_13 (Batc (None, 7, 42, 84, 128)    512       
_________________________________________________________________
activation_13 (Activation)   (None, 7, 42, 84, 128)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

# LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, mode='min', min_lr=0.00001)# write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/Project_data/train ; batch size = 32


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
21/21 [==============================] - 40s 2s/step - loss: 28.2297 - categorical_accuracy: 0.2563 - val_loss: 35.2595 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 35.25952, saving model to model_init_2021-07-1013_24_51.708723/model-00001-24.84453-0.29563-35.25952-0.23000.h5
Epoch 2/50
21/21 [==============================] - 37s 2s/step - loss: 19.3881 - categorical_accuracy: 0.4745 - val_loss: 25.6077 - val_categorical_accuracy: 0.2400

Epoch 00002: val_loss improved from 35.25952 to 25.60775, saving model to model_init_2021-07-1013_24_51.708723/model-00002-19.30325-0.48869-25.60775-0.24000.h5
Epoch 3/50
21/21 [==============================] - 36s 2s/step - loss: 19.1261 - categorical_accuracy: 0.4963 - val_loss: 24.4875 - val_categorical_accuracy: 0.2800

Epoch 00003: val_loss improved from 25.60775 to 24.48748, saving model to model_init_2021-07-1013_24_51.708723/model-00003-19.08177-0.51735-24.48748-0.28000.h5
Epoch 4/50
21/21 [======

## Model using transfer learning

In [ ]:
# from keras.models import Sequential, Model
# from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
# from keras.layers.convolutional import Conv3D, MaxPooling3D
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
# from keras import optimizers
# from keras.applications.vgg16 import VGG16

# #write your model here
# # Transfer Learning

# base_model = VGG16(include_top=False, weights='imagenet', input_shape=(84,84,3))
# x = base_model.output
# x = Flatten()(x)
# # x.add(Dropout(0.5))
# features = Dense(64, activation='relu')(x)
# conv_model = Model(inputs= base_model.input, outputs= features)

# for layer in base_model.layers:
#     layer.trainable = False

# # RNN
# model2=Sequential()
# model2.add(TimeDistributed(conv_model, input_shape=(15,84,84,3)))
# model2.add(GRU(32, return_sequences=True))
# model2.add(GRU(16))
# model2.add(Dropout(0.5))
# model2.add(Dense(8, activation='relu'))
# model2.add(Dense(5, activation='softmax'))

In [ ]:
# # optimiser = #write your optimizer
# sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.7, nesterov=True)
# model2.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# print (model2.summary())

In [ ]:
# # model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)